In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.fftpack



import wave
import struct
import sys
import csv

from scipy.io import wavfile
from scipy.signal import resample

import sounddevice as sd
import soundfile as sf

import os

from IPython import display
from IPython.display import Audio

#import tensorflow for yamnet
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio


data_folder = './data/'
wav_folder_rx ='./samples_A_RX1_C0/'
wav_folder_deformed ='./samples_A_DEFORMED1_C0/'

In [4]:
#CSV filenames
deformed01 = 'A_DEFORMED1_C0'
deformed02 = 'A_DEFORMED1_C0'
rx01 = 'A_RX1_C0'

#WAV filenames
wav_rx = 'sample_A_RX1_C00'
df = pd.read_csv(data_folder + deformed02 + '.csv', header = None)

In [5]:

#initialisation of clock variables
clock = 3048780.48780488
delay_in_samples = 304878
g_T = 1 / clock
g_N = len(df)

print("the frequency of the signal is", clock * 1e-6, "MHz")
print("the total duration of the signal is", g_N / clock, "seconds")
print("number of samples :", len(df))


the frequency of the signal is 3.04878048780488 MHz
the total duration of the signal is 10.409464743999994 seconds
number of samples : 31736173


In [8]:
#remove the last points so that the array length is divisible by the number of points per sample 
#in order to have equal lengths sample
def clean_data(df):
    epsilon =  1e-1
    #remove the silence at the end: 
    for i in range(len(df) - 1, 0, -1):
        if(df[0][i] > epsilon):
            print("deleted", (len(df) - i) / clock, "samples from the end because we assume it is silence from", len(df) / clock, "samples")
            return df[:i]

def split_csv(filename, T):
    df = pd.read_csv(data_folder + filename + '.csv', header = None)
    df = clean_data(df)
    df.rename(columns = {0:'amplitude'}, inplace = True)
    point_per_sample = int(160e-3 * clock)
    
    nb_of_samples = int(len(df)/point_per_sample)
    data = df[:nb_of_samples * point_per_sample]
    return np.array_split(data, nb_of_samples)

data_deformed = split_csv(deformed01, g_T)
#data_rx = split_csv(rx01, g_T)


deleted 1.2691172799999992 samples from the end because we assume it is silence from 10.409464743999994 samples


In [9]:
from scipy import signal

#Wavelets
t = np.linspace(-1, 1, 200, endpoint=False)
flat_list = []
widths = np.arange(1, 31)
for sublist in data_deformed:
    for item in sublist['amplitude']:
        flat_list.append(item)


# cwt has shape(30, 128795) 30 being widths
cwt = signal.cwt(flat_list, signal.ricker,widths)

In [14]:
#standardize data
normalized_cwt=(cwt-cwt.mean())/cwt.std()
print(normalized_cwt)



[[ 1.89715206e-03  8.17831480e-04 -2.01714086e-03 ...  3.49778435e-01
   3.63097780e+00  9.40086257e+00]
 [-5.86234753e-03  5.95608922e-03  1.70033451e-02 ...  9.84106609e+00
   1.24308497e+01  9.12124329e+00]
 [ 5.87897810e-04  1.56878155e-02  2.93046863e-02 ...  1.50447341e+01
   1.32753235e+01  7.85236807e+00]
 ...
 [-2.83781863e-02 -2.76508467e-02 -2.68712364e-02 ...  8.04028751e+00
   5.38709406e+00  2.71342071e+00]
 [-2.92518314e-02 -2.84240012e-02 -2.75411792e-02 ...  7.90592470e+00
   5.29678984e+00  2.66888301e+00]
 [-2.99748840e-02 -2.90545801e-02 -2.80775123e-02 ...  7.77812810e+00
   5.21100283e+00  2.62661995e+00]]


In [16]:
#PCA
from sklearn.decomposition import PCA
pca = PCA(len(normalized_cwt))
pca.fit(normalized_cwt)
train_img = pca.transform(normalized_cwt)
print(train_img)